In [1]:
"""!pip download pylibjpeg-libjpeg==2.2.0 -d /kaggle/working/mysitepackages/pylibjpeg_libjpeg
!pip download pylibjpeg-openjpeg==2.3.0 -d /kaggle/working/mysitepackages/pylibjpeg_openjpeg
!pip download scikit-learn==1.7.0 -d /kaggle/working/mysitepackages/scikit_learn
!pip download matplotlib==3.10.3 -d /kaggle/working/mysitepackages/matplotlib"""

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 32.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 80.0 MB/s eta 0:00:00:00:0100:01
Saved ./mysitepackages/pylibjpeg_libjpeg/pylibjpeg_libjpeg-2.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Saved ./mysitepackages/pylibjpeg_libjpeg/numpy-2.3.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl
Successfully downloaded pylibjpeg-libjpeg numpy
  Using cached numpy-2.3.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.8 MB/s eta 0:00:0000:010:01
Using cached numpy-2.3.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.9 MB)
Saved ./mysitepackages/pylibjpeg_openjpeg/pylibjpeg_openjpeg-2.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Saved ./mysitepackages/pylibjpeg_openjpeg/numpy-

In [2]:
"""from zipfile import ZipFile
import os

dirName = '/kaggle/working/mysitepackages'
zipName = 'dicom_tpu_supporting_combo.zip'

# Create a ZipFile Object
with ZipFile(zipName, 'w') as zipObj:
    # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk(dirName):
        for filename in filenames:
            if (filename != zipName):
                # create complete filepath of file in directory
                filePath = os.path.join(folderName, filename)
                # Add file to zip
                zipObj.write(filePath)"""

## This is the working combination

In [1]:
!pip install duckdb --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
!pip install python-gdcm
!pip install pylibjpeg
!pip install pylibjpeg-libjpeg==2.2.0 --find-links=/kaggle/input/suman-dicom-tpu-supporting-combo/kaggle/working/mysitepackages/pylibjpeg_libjpeg
!pip install pylibjpeg-openjpeg==2.3.0 --find-links=/kaggle/input/suman-dicom-tpu-supporting-combo/kaggle/working/mysitepackages/pylibjpeg_openjpeg
!pip install matplotlib==3.10.3 --no-index --find-links=/kaggle/input/suman-dicom-tpu-supporting-combo/kaggle/working/mysitepackages/matplotlib
!pip install scikit-learn==1.7.0 --no-index --find-links=/kaggle/input/suman-dicom-tpu-supporting-combo/kaggle/working/mysitepackages/scikit_learn
!pip install polars --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
!pip install pydicom

Looking in links: /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 77.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Looking in links: /kaggle/input/suman-dicom-tpu-supporting-combo/kaggle/working/mysitepackages/pylibjpeg_libjpeg
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 19.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Looking in links: /kaggle/input/suman-dicom-tpu-supporting-combo/kaggle/working/mysitepackages/pylibjpeg_openjpeg
     ━━━━━━━━━━━━━

In [2]:
!pip install tensorflow-io==0.37.1 --no-index --find-links=/kaggle/input/suman-tpu-working-combo/kaggle/working/mysitepackages/tensorflow_io
!pip install tensorflow==2.16.1 --find-links=/kaggle/input/suman-tpu-working-combo/kaggle/working/mysitepackages/tensorflow
!pip install tensorflow-tpu==2.16.1 --find-links=/kaggle/input/suman-tpu-working-combo/kaggle/working/mysitepackages/tensorflow_tpu

Looking in links: /kaggle/input/suman-tpu-working-combo/kaggle/working/mysitepackages/tensorflow_io
Looking in links: /kaggle/input/suman-tpu-working-combo/kaggle/working/mysitepackages/tensorflow
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 908.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 70.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 81.2 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: tensorboard
    Found existing i

In [3]:
import pydicom
from pydicom import dcmread
from pydicom.dataset import FileDataset, FileMetaDataset
from pydicom.uid import generate_uid, ImplicitVRLittleEndian

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
import pickle
import gc
import ctypes
from pathlib import Path
import logging
import json
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor
import datetime
from typing import Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras

E0000 00:00:1759769091.341204      10 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D1006 16:44:51.347976343      10 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1006 16:44:51.347987243      10 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1006 16:44:51.347989703      10 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1006 16:44:51.347991253      10 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1006 16:44:51.347992593      10 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

In [4]:
print(tf.__version__)
print(tfio.__version__)

2.16.1
0.37.1


In [5]:
# detect TPUs
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

print("Number of accelerators: ", tpu_strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1759769111.221989      10 service.cc:145] XLA service 0x57c1d9943720 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1759769111.222031      10 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1759769111.222034      10 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1759769111.222036      10 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1759769111.222038      10 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1759769111.222040      10 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1759769111.222041      10 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1759769111.222043      10 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1759769111.222044      10 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [6]:
pl.Config(fmt_str_lengths=1000)
pl.Config.set_tbl_rows(1000)

polars.config.Config

In [7]:
allowed_tags = ['BitsAllocated', 'BitsStored', 'Rows', 'Columns', 'FrameOfReferenceUID', 'HighBit', 'ImageOrientationPatient'
                , 'ImagePositionPatient', 'InstanceNumber', 'Modality', 'PhotometricInterpretation'
                , 'PixelRepresentation', 'PixelSpacing', 'PlanarConfiguration', 'RescaleIntercept', 'RescaleSlope'
                , 'RescaleType', 'SamplesPerPixel', 'SliceThickness', 'SpacingBetweenSlices']

In [8]:
df_all_data = pl.read_parquet('/kaggle/input/rsna-aneurysm-train-metadata-suman/full_training_data_v2.parquet')
print("Shape of training metadata", df_all_data.shape)
df_all_data.columns

Shape of training metadata (1000771, 28)


['aneurysm_position',
 'file_name',
 'file_path',
 'image_shape',
 'coordinates_x',
 'coordinates_y',
 'coordinates_f',
 'min_max_diff',
 'aneurysm_present_in_series',
 'aneurysm_present_in_image',
 'seriesinstanceuid',
 'patientage',
 'patientsex',
 'modality',
 'left_infraclinoid_internal_carotid_artery',
 'right_infraclinoid_internal_carotid_artery',
 'left_supraclinoid_internal_carotid_artery',
 'right_supraclinoid_internal_carotid_artery',
 'left_middle_cerebral_artery',
 'right_middle_cerebral_artery',
 'anterior_communicating_artery',
 'left_anterior_cerebral_artery',
 'right_anterior_cerebral_artery',
 'left_posterior_communicating_artery',
 'right_posterior_communicating_artery',
 'basilar_tip',
 'other_posterior_circulation',
 'aneurysm_position_encoded']

In [16]:
dd.sql(" \
select t1.modality, t1.aneurysm_present_in_image, t1.per_mod_count, \
round(t1.per_mod_count/t2.total_count ,3) as modality_pct \
from \
( \
select modality, aneurysm_present_in_image, cast(count(1) as float) as per_mod_count from df_all_data \
group by modality, aneurysm_present_in_image \
)t1 \
join \
(select cast(count(1) as float) as total_count from df_all_data)t2 \
on 1=1 \
order by 1 \
").pl()

modality,aneurysm_present_in_image,per_mod_count,modality_pct
str,i32,f32,f32
"""CTA""",0,725615.0,0.725
"""CTA""",1,1220.0,0.001
"""MRA""",0,195785.0,0.196
"""MRA""",1,659.0,0.001
"""MRI T1post""",0,47046.0,0.047
"""MRI T1post""",1,93.0,0.0
"""MRI T2""",0,30073.0,0.03
"""MRI T2""",1,280.0,0.0


In [9]:
def extract_single_frame(multiframe_path, slice_number, output_path=None):
    """
    Extract a single frame from a multi-frame DICOM
    
    Args:
        multiframe_path: Path to multi-frame DICOM file
        slice_number: The slice number to extract (0-based index)
        output_path: Path to save the single-frame DICOM. If None, returns the dataset
    """
    try:
        # Read the multi-frame DICOM with force=True to handle potentially corrupted files
        multi_ds = dcmread(multiframe_path, force=True)
        
        # Verify it's a multi-frame image
        if not hasattr(multi_ds, 'NumberOfFrames'):
            raise ValueError("Input DICOM is not a multi-frame image")
        
        # Check if slice number is valid
        if slice_number >= multi_ds.NumberOfFrames:
            raise ValueError(f"Slice number {slice_number} is out of range. "
                           f"Image has {multi_ds.NumberOfFrames} frames")
        
        # Create new dataset for single frame
        single_ds = FileDataset(output_path or "temp.dcm", {}, 
                              file_meta=FileMetaDataset(), 
                              preamble=b"\0" * 128)
        
        # Copy attributes from multi-frame dataset
        attrs_to_copy = allowed_tags
        
        for attr in attrs_to_copy:
            if hasattr(multi_ds, attr):
                setattr(single_ds, attr, getattr(multi_ds, attr))
        
        # Generate new UIDs
        single_ds.SOPInstanceUID = generate_uid()
        single_ds.file_meta.MediaStorageSOPInstanceUID = single_ds.SOPInstanceUID
        
        # Set transfer syntax to uncompressed little endian
        single_ds.file_meta.TransferSyntaxUID = ImplicitVRLittleEndian
        single_ds.file_meta.MediaStorageSOPClassUID = multi_ds.file_meta.MediaStorageSOPClassUID
        if hasattr(multi_ds.file_meta, 'ImplementationClassUID'):
            single_ds.file_meta.ImplementationClassUID = multi_ds.file_meta.ImplementationClassUID
        
        # Set instance-specific attributes
        single_ds.InstanceNumber = slice_number + 1
        
        try:
            # Try to get pixel array directly
            pixel_array = multi_ds.pixel_array[slice_number]
        except Exception as e:
            #print(f"Warning: Could not directly access pixel_array: {e}")
            # Alternative approach: decompress and get pixels
            if hasattr(multi_ds, 'decompress'):
                multi_ds.decompress()
            pixel_array = multi_ds.pixel_array[slice_number]
        
        # Set pixel data
        single_ds.PixelData = pixel_array.tobytes()
        
        # Update image-specific attributes
        single_ds.NumberOfFrames = 1
        
        # Try to copy position and orientation
        try:
            if hasattr(multi_ds, 'PerFrameFunctionalGroupsSequence'):
                frame_content = multi_ds.PerFrameFunctionalGroupsSequence[slice_number]
                
                if hasattr(frame_content, 'PlanePositionSequence'):
                    position = frame_content.PlanePositionSequence[0].ImagePositionPatient
                    single_ds.ImagePositionPatient = position
                
                if hasattr(frame_content, 'PlaneOrientationSequence'):
                    orientation = frame_content.PlaneOrientationSequence[0].ImageOrientationPatient
                    single_ds.ImageOrientationPatient = orientation
        except Exception as e:
            #print(f"Warning: Could not copy position/orientation: {e}")
            raise
        
        # Add creation timestamp
        dt = datetime.datetime.now()
        single_ds.ContentDate = dt.strftime('%Y%m%d')
        single_ds.ContentTime = dt.strftime('%H%M%S.%f')
        
        # Save or return the dataset
        if output_path:
            single_ds.save_as(output_path, write_like_original=False)
            return None
        return single_ds
    
    except Exception as e:
        #print(f"Error extracting frame: {e}")
        raise

# Alternative version using different approach for compressed files
def extract_single_frame_alternative(multiframe_path, slice_number, output_path=None):
    """
    Alternative version for handling problematic files
    """
    try:
        # Read with force and stop before pixels
        multi_ds = dcmread(multiframe_path, force=True, stop_before_pixels=True)
        
        # Read pixel data separately
        with open(multiframe_path, 'rb') as f:
            multi_ds.PixelData = f.read()
        
        # Decompress if needed
        if hasattr(multi_ds, 'decompress'):
            multi_ds.decompress()
        
        # Get pixel array
        pixel_array = multi_ds.pixel_array[slice_number]
        
        # Create new dataset
        single_ds = FileDataset(output_path or "temp.dcm", {}, 
                              file_meta=FileMetaDataset(), 
                              preamble=b"\0" * 128)
        
        # Copy attributes (same as before)
        attrs_to_copy = allowed_tags
        
        for attr in attrs_to_copy:
            if hasattr(multi_ds, attr):
                setattr(single_ds, attr, getattr(multi_ds, attr))
        
        # Generate new UIDs
        single_ds.SOPInstanceUID = generate_uid()
        single_ds.file_meta.MediaStorageSOPInstanceUID = single_ds.SOPInstanceUID
        
        # Set transfer syntax to uncompressed little endian
        single_ds.file_meta.TransferSyntaxUID = ImplicitVRLittleEndian
        single_ds.file_meta.MediaStorageSOPClassUID = multi_ds.file_meta.MediaStorageSOPClassUID
        if hasattr(multi_ds.file_meta, 'ImplementationClassUID'):
            single_ds.file_meta.ImplementationClassUID = multi_ds.file_meta.ImplementationClassUID
        
        # Set instance-specific attributes
        single_ds.InstanceNumber = slice_number + 1
        
        try:
            # Try to get pixel array directly
            pixel_array = multi_ds.pixel_array[slice_number]
        except Exception as e:
            #print(f"Warning: Could not directly access pixel_array: {e}")
            # Alternative approach: decompress and get pixels
            if hasattr(multi_ds, 'decompress'):
                multi_ds.decompress()
            pixel_array = multi_ds.pixel_array[slice_number]
        
        # Set pixel data
        single_ds.PixelData = pixel_array.tobytes()
        
        # Update image-specific attributes
        single_ds.NumberOfFrames = 1
        
        # Try to copy position and orientation
        try:
            if hasattr(multi_ds, 'PerFrameFunctionalGroupsSequence'):
                frame_content = multi_ds.PerFrameFunctionalGroupsSequence[slice_number]
                
                if hasattr(frame_content, 'PlanePositionSequence'):
                    position = frame_content.PlanePositionSequence[0].ImagePositionPatient
                    single_ds.ImagePositionPatient = position
                
                if hasattr(frame_content, 'PlaneOrientationSequence'):
                    orientation = frame_content.PlaneOrientationSequence[0].ImageOrientationPatient
                    single_ds.ImageOrientationPatient = orientation
        except Exception as e:
            #print(f"Warning: Could not copy position/orientation: {e}")
            raise
        
        # Add creation timestamp
        dt = datetime.datetime.now()
        single_ds.ContentDate = dt.strftime('%Y%m%d')
        single_ds.ContentTime = dt.strftime('%H%M%S.%f')
        
        # Save or return the dataset
        if output_path:
            single_ds.save_as(output_path, write_like_original=False)
            return None
        return single_ds
        
    except Exception as e:
        #print(f"Error in alternative extraction: {e}")
        raise

# Function to try both methods
def safe_extract_single_frame(multiframe_path, slice_number, output_path=None):
    """
    Try both extraction methods
    """
    try:
        return extract_single_frame(multiframe_path, slice_number, output_path)
    except Exception as e:
        #print(f"Primary method failed: {e}")
        #print("Trying alternative method...")
        try:
            return extract_single_frame_alternative(multiframe_path, slice_number, output_path)
        except Exception as e2:
            #print(f"Alternative method also failed: {e2}")
            raise

# Version with zoom functionality
def load_and_view_single_slice_with_zoom(dcm_path, x_coord, y_coord, f_coord=None, zoom_size=100):
    """
    Load and display a single DICOM slice with crosshair and zoomed inset
    
    Args:
        dcm_path: Path to the DICOM file
        x_coord: x coordinate for the crosshair
        y_coord: y coordinate for the crosshair
        zoom_size: Size of the zoom window in pixels
    """
    # Read DICOM file
    if f_coord:
        ds = safe_extract_single_frame(dcm_path, f_coord)
    else:
        ds = dcmread(dcm_path)
    img = ds.pixel_array
    
    # Create figure and axes
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
    
    # Main image with crosshair
    ax1.imshow(img, cmap='gray')
    ax1.axvline(x=x_coord, color='red', alpha=0.5)
    ax1.axhline(y=y_coord, color='red', alpha=0.5)
    ax1.plot(x_coord, y_coord, 'r+', markersize=10, markeredgewidth=2)
    
    # Zoomed region
    x_start = int(max(0, x_coord - zoom_size/2))
    x_end = int(min(img.shape[1], x_coord + zoom_size/2))
    y_start = int(max(0, y_coord - zoom_size/2))
    y_end = int(min(img.shape[0], y_coord + zoom_size/2))
    
    zoomed = img[y_start:y_end, x_start:x_end]
    ax2.imshow(zoomed, cmap='gray')
    
    # Add crosshair to zoomed region
    center_x = x_coord - x_start
    center_y = y_coord - y_start
    ax2.axvline(x=center_x, color='red', alpha=0.5)
    ax2.axhline(y=center_y, color='red', alpha=0.5)
    ax2.plot(center_x, center_y, 'r+', markersize=10, markeredgewidth=2)
    
    ax1.axis('off')
    ax2.axis('off')
    ax1.set_title('Full Image')
    ax2.set_title('Zoomed Region')
    
    plt.tight_layout()
    plt.show()

## Checking the imbalanceness of the data

In [10]:
target_variables = ['left_infraclinoid_internal_carotid_artery',  'right_infraclinoid_internal_carotid_artery'
                    ,  'left_supraclinoid_internal_carotid_artery',  'right_supraclinoid_internal_carotid_artery'
                    ,  'left_middle_cerebral_artery',  'right_middle_cerebral_artery'
                    ,  'anterior_communicating_artery',  'left_anterior_cerebral_artery'
                    ,  'right_anterior_cerebral_artery',  'left_posterior_communicating_artery'
                    ,  'right_posterior_communicating_artery'
                    ,  'basilar_tip',  'other_posterior_circulation'
                    ,  'no_aneurysm'
                   ]

In [11]:
label_cols = ['left_infraclinoid_internal_carotid_artery',  'right_infraclinoid_internal_carotid_artery'
                    ,  'left_supraclinoid_internal_carotid_artery',  'right_supraclinoid_internal_carotid_artery'
                    ,  'left_middle_cerebral_artery',  'right_middle_cerebral_artery'
                    ,  'anterior_communicating_artery',  'left_anterior_cerebral_artery'
                    ,  'right_anterior_cerebral_artery',  'left_posterior_communicating_artery'
                    ,  'right_posterior_communicating_artery'
                    ,  'basilar_tip',  'other_posterior_circulation', 'aneurysm_present_in_image'
                   ]

In [12]:
def process_mri_t2(image: np.ndarray):
    """
    Specific preprocessing for T2-weighted MRI
    - Bias field correction
    - Intensity normalization
    """
    # Convert to float32
    image = image.astype(np.float32)
    
    # Apply N4 bias field correction if needed
    # Note: This would require SimpleITK, commented out for simplicity
    # image = self._apply_bias_correction(image)
    
    # Z-score normalization
    mean = np.mean(image)
    std = np.std(image)
    image = (image - mean) / (std + 1e-10)
    
    # Scale to [0,1] range
    image = (image - np.min(image)) / (np.max(image) - np.min(image) + 1e-10)
    
    return image

def process_mri_t1_post(image: np.ndarray):
    """
    Specific preprocessing for T1 post-contrast MRI
    - Enhancement of contrast regions
    - Intensity standardization
    """
    image = image.astype(np.float32)
    
    # Enhance contrast
    p2, p98 = np.percentile(image, (2, 98))
    image = np.clip(image, p2, p98)
    
    # Normalize to [0,1]
    image = (image - p2) / (p98 - p2 + 1e-10)
    
    return image

def process_cta(image: np.ndarray, dcm: pydicom.dataset.FileDataset):
    """
    Specific preprocessing for CTA
    - Window/level optimization for vessel visualization
    - Vessel enhancement
    """
    # Get window settings (typical for CTA)
    window_center = 100  # Typical for CTA
    window_width = 700   # Typical for CTA
    
    # Override with DICOM values if available
    if hasattr(dcm, 'WindowCenter') and hasattr(dcm, 'WindowWidth'):
        window_center = dcm.WindowCenter
        window_width = dcm.WindowWidth
        
        # Handle multiple window settings
        if isinstance(window_center, pydicom.multival.MultiValue):
            window_center = window_center[0]
        if isinstance(window_width, pydicom.multival.MultiValue):
            window_width = window_width[0]
    
    # Apply window/level
    min_value = window_center - window_width // 2
    max_value = window_center + window_width // 2
    image = np.clip(image, min_value, max_value)
    
    # Normalize to [0,1]
    image = (image - min_value) / (max_value - min_value + 1e-10)
    
    return image

def process_mra(image: np.ndarray):
    """
    Specific preprocessing for MRA
    - Maximum intensity projection consideration
    - Vessel enhancement
    """
    image = image.astype(np.float32)
    
    # Normalize based on tissue characteristics
    p0, p100 = np.percentile(image, (0, 100))
    image = (image - p0) / (p100 - p0 + 1e-10)
    
    # Optional: Vessel enhancement using Frangi filter
    # Note: This would require additional dependencies
    # image = self._apply_vessel_enhancement(image)
    
    return image

In [13]:
def read_and_parse_dicom_files_tensorflow_train_cta(dcm_path, f_coord=None):
    #print("dcm_path - ", dcm_path)
    raw_image = tf.io.read_file(dcm_path)
    image = tfio.image.decode_dicom_image(raw_image, scale='auto', on_error='lossy', dtype=tf.float32)

    tag_id = tfio.image.dicom_tags.Modality
    tag_value = tfio.image.decode_dicom_data(raw_image,tag_id)
    modality = tag_value.numpy().decode('UTF-8')

    if modality == 'CTA':
        image = process_cta(image, dcm_path)
        
    if modality == 'MRA':
        image = process_mra(image)
        
    if modality == 'MRI T2':
        image = process_mri_t2(image)

    if modality == 'MRI T1post':
        image = process_mri_t1_post(image)

    if np.any(np.isnan(image)):
        print(f"Warning: NaN values found in image {dcm_path}")
        
    if np.any(np.isinf(image)):
        print(f"Warning: Infinite values found in image {dcm_path}")
    
    specific_slice_tensor = tf.slice(
        image,
        begin=[f_coord, 0, 0, 0],
        size=[1, image.shape[1], image.shape[2], image.shape[3]]
    )
    # Squeeze to remove the extra dimension of size 1
    specific_slice_tensor = tf.squeeze(specific_slice_tensor)
    
    expanded_image = tf.expand_dims(specific_slice_tensor, -1)
    m, M=tf.math.reduce_min(expanded_image), tf.math.reduce_max(expanded_image)
    expanded_image = (tf.image.grayscale_to_rgb(expanded_image)-m)/(M-m)
    expanded_image = tf.image.resize(expanded_image, (128,128))
    sqzd_image = tf.squeeze(expanded_image)
    return sqzd_image

def load_dicom_tf_cta(filepath_tensor, f_coord_tensor=None):
    """
    Wrapper function to call read_and_parse_dicom_files_tensorflow_train within a tf.data.Dataset pipeline.
    """
    #if f_coord_tensor:
    image = tf.py_function(
        read_and_parse_dicom_files_tensorflow_train_cta,
        inp=[filepath_tensor, f_coord_tensor],
        Tout=tf.float32 # Adjust Tout based on the expected output type of your image data
    )
    """else:
        image = tf.py_function(
            read_and_parse_dicom_files_tensorflow_train,
            inp=[filepath_tensor],
            Tout=tf.float32 # Adjust Tout based on the expected output type of your image data
    )"""
    # Ensure the shape is defined if known, as tf.py_function can lose shape info
    #image.set_shape([128, 128]) # Example for a 2D image, adjust as needed
    return image 

def preprocessing_cta(dcm_path, f_coord):
    train_img = load_dicom_tf_cta(dcm_path, f_coord)
    #train_img = read_and_parse_dicom_files_tensorflow_train(dcm_path, f_coord)
    train_img = tf.reshape(train_img, shape=(128, 128, 3))
    return train_img

def load_dataset_tensorflow_train_cta(dcm_path, f_coord, labels):
    image = preprocessing_cta(dcm_path, f_coord)
    return {"images": tf.cast(image, tf.float32), "labels": tf.cast(labels, tf.float32)}

def dict_to_tuple(inputs):
    return inputs["images"], inputs["labels"]

In [14]:
cols_to_select = ['file_path', 'coordinates_f', 'aneurysm_position_encoded'] + label_cols

df_for_train_baseline = df_all_data.select(cols_to_select)

In [15]:
df_for_train_baseline.head(3)

file_path,coordinates_f,aneurysm_position_encoded,left_infraclinoid_internal_carotid_artery,right_infraclinoid_internal_carotid_artery,left_supraclinoid_internal_carotid_artery,right_supraclinoid_internal_carotid_artery,left_middle_cerebral_artery,right_middle_cerebral_artery,anterior_communicating_artery,left_anterior_cerebral_artery,right_anterior_cerebral_artery,left_posterior_communicating_artery,right_posterior_communicating_artery,basilar_tip,other_posterior_circulation,aneurysm_present_in_image
str,i32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i32
"""/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.31376096656152917095020145403163300782/1.2.826.0.1.3680043.8.498.75688647949837735500393600700770738400.dcm""",0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.31407800911736473408414897110314879799/1.2.826.0.1.3680043.8.498.13230044731894805015461835474149549749.dcm""",0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.31443768154622563276090376579664022056/1.2.826.0.1.3680043.8.498.45049134008296770227979116448060171856.dcm""",0,6,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [16]:
x_for_train, x_holdout = train_test_split(df_for_train_baseline, test_size=0.5, random_state=42)
x_train, x_for_valid = train_test_split(x_for_train, test_size=0.4, random_state=42)
x_test, x_valid = train_test_split(x_for_valid, test_size=0.2, random_state=42)

print("********* CTA ****************")
print("Holdout data shape : {0}".format(x_holdout.shape))
print("Training data shape : {0}".format(x_train.shape))
print("Test data shape : {0}".format(x_test.shape))
print("Validation data shape : {0}".format(x_valid.shape))

********* CTA ****************
Holdout data shape : (500386, 17)
Training data shape : (300231, 17)
Test data shape : (160123, 17)
Validation data shape : (40031, 17)


In [17]:
x_train.columns

['file_path',
 'coordinates_f',
 'aneurysm_position_encoded',
 'left_infraclinoid_internal_carotid_artery',
 'right_infraclinoid_internal_carotid_artery',
 'left_supraclinoid_internal_carotid_artery',
 'right_supraclinoid_internal_carotid_artery',
 'left_middle_cerebral_artery',
 'right_middle_cerebral_artery',
 'anterior_communicating_artery',
 'left_anterior_cerebral_artery',
 'right_anterior_cerebral_artery',
 'left_posterior_communicating_artery',
 'right_posterior_communicating_artery',
 'basilar_tip',
 'other_posterior_circulation',
 'aneurysm_present_in_image']

## Getting the biases

In [18]:
def get_bias(df):
    df_agg = df.group_by(["aneurysm_position_encoded"]).agg(pl.col("file_path").count())
    list_of_positions = pl.Series(df_agg.select(pl.col('aneurysm_position_encoded'))).to_list()
    bias_arr_prep_dict = {}
    
    for pos in list_of_positions:
        bias_arr_prep_dict[pos] = df_agg.filter(pl.col('aneurysm_position_encoded')==pos).select(pl.col('file_path')).item(0, 0)
    
    total_pos = 0
    for pos in list_of_positions:
        total_pos = total_pos + bias_arr_prep_dict[pos]

    bias_dict = {}
    bias_list = []
    for pos in list_of_positions:
        local_pos = total_pos - bias_arr_prep_dict[pos]
        bias_dict[pos] = round(np.log(bias_arr_prep_dict[pos]/local_pos),5)
        bias_list.append(bias_dict[pos])

    return bias_dict, bias_list

In [19]:
bias_dict, bias_list = get_bias(x_train)

print(bias_dict)
print(bias_list)
print(len(bias_list))

{3: -9.21101, 0: -7.87573, 6: -7.83279, 9: -9.83967, 12: -9.43417, 4: -8.16937, 1: -9.43417, 7: 6.085, 10: -9.24492, 13: -8.20532, 2: -9.83967, 5: -9.14646, 8: -9.24492, 11: -8.20532}
[-9.21101, -7.87573, -7.83279, -9.83967, -9.43417, -8.16937, -9.43417, 6.085, -9.24492, -8.20532, -9.83967, -9.14646, -9.24492, -8.20532]
14


In [20]:
np.array(bias_list)

array([-9.21101, -7.87573, -7.83279, -9.83967, -9.43417, -8.16937,
       -9.43417,  6.085  , -9.24492, -8.20532, -9.83967, -9.14646,
       -9.24492, -8.20532])

In [18]:
def generate_tf_datasets(p_df, p_BATCH_SIZE_PER_REPLICA):

    BATCH_SIZE = p_BATCH_SIZE_PER_REPLICA * tpu_strategy.num_replicas_in_sync
    
    image_filenames = pl.Series(p_df.select(pl.col('file_path'))).to_list()
    image_f_coords = pl.Series(p_df.select(pl.col('coordinates_f'))).fill_null(0).to_list()
    image_labels = pl.Series(p_df.select(label_cols)).map_elements(lambda d: list(d.values())).to_list()
    #image_labels = pl.Series(p_df.select(pl.col('aneurysm_position_encoded'))).to_list()
    
    image_dataset = tf.data.Dataset.from_tensor_slices((image_filenames, image_f_coords, image_labels))
    
    image_ds = image_dataset.map(load_dataset_tensorflow_train_cta, num_parallel_calls=tf.data.AUTOTUNE)
    image_ds = image_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
    image_ds = image_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
    image_ds = image_ds.prefetch(tf.data.AUTOTUNE)
    
    return image_ds

## Check for NaNs/Infs in the tensors

In [40]:
def validate_training_data(images, labels):
    """
    Validate training data before model training
    """
    
    # Check images
    if (np.any(np.isnan(images))) or (np.any(np.isinf(images))):
        print("\n=== Data Validation Report from images ===")
        print("\nImage Data:")
        print(f"Shape: {images.shape}")
        #print("dcm_paths - ", dcm_paths)
        print(f"Value range: [{np.min(images)}, {np.max(images)}]")
        print(f"Contains NaN: {np.any(np.isnan(images))}")
        print(f"Contains Inf: {np.any(np.isinf(images))}")
    else:
        pass
    
    # Check labels
    if (np.any(np.isnan(labels))):
        print("\n=== Data Validation Report from labels ===")
        print("\nLabels:")
        print(f"Shape: {labels.shape}")
        print(f"Unique values: {np.unique(labels)}")
        print(f"Contains NaN: {np.any(np.isnan(labels))}")
    else:
        pass

In [19]:
train_ds = generate_tf_datasets(p_df=x_train, p_BATCH_SIZE_PER_REPLICA = 16)
test_ds = generate_tf_datasets(p_df=x_test, p_BATCH_SIZE_PER_REPLICA = 16)
valid_ds = generate_tf_datasets(p_df=x_valid, p_BATCH_SIZE_PER_REPLICA = 16)

In [41]:
for batch_features, batch_labels in train_ds:
    print(f"Features: {batch_features.numpy().shape}, Labels: {batch_labels.numpy().shape}")
    for i in range(batch_features.numpy().shape[0]):
        validate_training_data(batch_features.numpy()[i],batch_labels.numpy()[i])

W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers t

Features: (128, 128, 128, 3), Labels: (128,)


W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stor

Features: (128, 128, 128, 3), Labels: (128,)


W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit positio

...

W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit positio

Features: (128, 128, 128, 3), Labels: (128,)

W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (

W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value


KeyboardInterrupt: 

## Define the custom competition metric

In [20]:
class_weights = [1,1,1,1,1,1,1,13,1,1,1,1,1,1]
weights_array = np.asarray(class_weights)
weights_array = weights_array / np.sum(weights_array)

"""def mwc_roc_auc(y_true, y_pred):
    individual_aucs = roc_auc_score(y_true, y_pred, average=None)
    return tf.reduce_sum((individual_aucs * weights_array), 0).numpy()"""
    #return np.sum(individual_aucs * weights_array)

class CustomMetricMWCROC(tf.keras.metrics.Metric):
    def __init__(self, name='mwc_roc_auc', **kwargs):
        super().__init__(name=name, **kwargs)
        self.individual_aucs = self.add_weight(name='individual_aucs', initializer=)
        self.auc_sum = self.add_weight(name='auc_sum', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # y_true and y_pred are symbolic tensors
        individual_aucs = roc_auc_score(y_true, y_pred, average=None)
        self.auc_sum.assign_add(tf.reduce_sum((individual_aucs * weights_array), 0))

    def result(self):
        return self.auc_sum

    def reset_state(self):
        self.auc_sum.assign(0.0)

In [25]:
#from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers

with tpu_strategy.scope():
    
    rsna_input = layers.Input(shape=(128,128,3), name="rsna_input")
    #rsna_input = layers.Input(shape=(None,None,3), name="rsna_input")
    # Resize images to 299x299 before passing them to the model
    #resized_images = layers.Lambda(lambda x: tf.image.resize(x, (299, 299)))(rsna_input)

    #conv_base = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=rsna_input)
    conv_base = InceptionV3(include_top=False, weights="imagenet", input_tensor=rsna_input, input_shape=(128,128,3))
    #conv_base = InceptionV3(include_top=False, weights="imagenet", input_tensor=resized_images)
    conv_base.trainable = False

    #max_pool_layer_0 = layers.MaxPooling2D(name="max_pool_0", pool_size=(2, 2), strides=(1, 1), padding="same")(conv_base.output)
    #max_pool_layer_0 = layers.BatchNormalization()(conv_base.output)

    #conv2d_1 = layers.Conv2D(filters=100, kernel_size=4, strides=1, padding="same", activation="relu")(max_pool_layer_0)
    #max_pool_layer_1 = layers.MaxPooling2D(name="max_pool_1", pool_size=(2, 2), strides=(1, 1), padding="same")(conv2d_1)
    #max_pool_layer_1 = layers.BatchNormalization()(max_pool_layer_1)

    flattened_layer = layers.Flatten()(conv_base.output)

    #hidden_layer1 = layers.Dense(200, activation="relu", kernel_initializer=keras.initializers.LecunNormal(seed=None))(flattened_layer)
    #hidden_layer1 = layers.BatchNormalization()(hidden_layer1)
    #hidden_layer2 = layers.LeakyReLU(alpha=0.001)(hidden_layer1) 
    #hidden_layer2 = layers.Dense(100, activation="selu", kernel_initializer=keras.initializers.LecunNormal(seed=None))(hidden_layer1)
    #hidden_layer2 = layers.BatchNormalization()(hidden_layer2)
    #hidden_layer3 = layers.Dense(50, activation="selu", kernel_initializer=keras.initializers.LecunNormal(seed=None))(hidden_layer2)
    #hidden_layer3 = layers.BatchNormalization()(hidden_layer3)
    #rsna_output = layers.Dense(14, activation="softmax", bias_initializer=tf.keras.initializers.Constant(np.array(bias_list)))(flattened_layer)
    rsna_output = layers.Dense(14, activation="softmax")(flattened_layer)
    model1 = tf.keras.Model(rsna_input, rsna_output)

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("keras_effnet_cta.keras")
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
    tensor_board_cb = tf.keras.callbacks.TensorBoard(log_dir='/kaggle/working/logs', histogram_freq=1)

    callbacks_list = [checkpoint_cb, early_stopping_cb, tensor_board_cb]

    model_optimizer = tf.keras.optimizers.Adamax(
        learning_rate=0.00001,
        clipnorm=1.0
    )

    #model1.compile(loss="sparse_categorical_crossentropy", optimizer=model_optimizer, metrics=["accuracy"])
    #model1.compile(loss="sparse_categorical_crossentropy", optimizer=model_optimizer, metrics=[mwc_roc_auc])
    model1.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), optimizer=model_optimizer
                   , metrics=[tf.keras.metrics.AUC(multi_label=True, num_labels=14, from_logits=False
                                                   , label_weights=[1,1,1,1,1,1,1,1,1,1,1,1,1,13])])

In [46]:
model1.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ rsna_input          │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_846 (Conv2D) │ (None, 63, 63,    │        864 │ rsna_input[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 63, 63,    │         96 │ conv2d_846[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_846      │ (None, 63, 63,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_847 (Conv2D) │ (None, 61, 61,    │      9,216 │ activation_846[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 61, 61,    │         96 │ conv2d_847[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_847      │ (None, 61, 61,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_848 (Conv2D) │ (None, 61, 61,    │     18,432 │ activation_847[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 61, 61,    │        192 │ conv2d_848[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_848      │ (None, 61, 61,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_36    │ (None, 30, 30,    │          0 │ activation_848[0… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_849 (Conv2D) │ (None, 30, 30,    │      5,120 │ max_pooling2d_36… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 30, 30,    │        240 │ conv2d_849[0][0]  │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_849      │ (None, 30, 30,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_850 (Conv2D) │ (None, 28, 28,    │    138,240 │ activation_849[0… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 28,    │        576 │ conv2d_850[0][0]  │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_850      │ (None, 28, 28,    │          0 │ batch_normalizat

 Total params: 21,917,486 (83.61 MB)

 Trainable params: 114,702 (448.05 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [ ]:
with tpu_strategy.scope():
    history = model1.fit(train_ds, epochs=10, validation_data=valid_ds, callbacks=callbacks_list)

W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may crea

Epoch 1/10


I0000 00:00:1759769701.469369    1407 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(ee37377b72afe909:0:0), session_name()
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpe

   1/2345 ━━━━━━━━━━━━━━━━━━━━ 8:34:03 13s/step - auc_1: 0.0000e+00 - loss: 0.0000e+00

W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result


...

W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result


 484/2345 ━━━━━━━━━━━━━━━━━━━━ 15:36 503ms/step - auc_1: 0.4530 - loss: 0.0187

W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value


 485/2345 ━━━━━━━━━━━━━━━━━━━━ 15:36 504ms/step - auc_1: 0.4531 - loss: 0.0187

W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result


In [27]:
with tpu_strategy.scope():
    model1.save("rsna_aneurysm_keras_base.h5")

In [28]:
with tpu_strategy.scope():
    model1.evaluate(test_ds)

1817/1817 ━━━━━━━━━━━━━━━━━━━━ 884s 486ms/step - accuracy: 0.9980 - loss: 0.0190


In [29]:
def read_and_parse_dicom_files_tensorflow_infer(dcm_path, f_coord=None):
    #tf.config.run_functions_eagerly(True)
    raw_image = tf.io.read_file(dcm_path)
    image = tfio.image.decode_dicom_image(raw_image, scale='auto', dtype=tf.float32)
    image = process_cta(image, dcm_path)
    #if f_coord:
    specific_slice_tensor = tf.slice(
        image,
        begin=[f_coord, 0, 0, 0],
        size=[1, image.shape[1], image.shape[2], image.shape[3]]
    )
    # Squeeze to remove the extra dimension of size 1
    specific_slice_tensor = tf.squeeze(specific_slice_tensor)
    """else:
        specific_slice_tensor = tf.squeeze(image)"""
    #print("specific_slice_tensor - ", specific_slice_tensor.shape)
    expanded_image = tf.expand_dims(specific_slice_tensor, -1)
    m, M=tf.math.reduce_min(expanded_image), tf.math.reduce_max(expanded_image)
    expanded_image = (tf.image.grayscale_to_rgb(expanded_image)-m)/(M-m)
    expanded_image = tf.image.resize(expanded_image, (128,128))
    sqzd_image = tf.squeeze(expanded_image)
    #print("sqzd_image -", sqzd_image.shape)
    return sqzd_image

In [30]:
image_filenames = pl.Series(x_valid_cta.select(pl.col('file_path'))).to_list()[0]
image_f_coords = pl.Series(x_valid_cta.select(pl.col('coordinates_f'))).fill_null(0).to_list()[0]
#image_labels = pl.Series(x_valid_cta.select(label_cols)).map_elements(lambda d: list(d.values())).to_list()[0]
image_labels = pl.Series(x_valid_cta.select(pl.col('aneurysm_position_encoded'))).to_list()[0]

In [31]:
image_filenames

'/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.11424791872144105045252084806718835697/1.2.826.0.1.3680043.8.498.39036260502558085256333876017282342878.dcm'

In [33]:
test_image = load_dicom_tf_cta(image_filenames, image_f_coords)
#read_and_parse_dicom_files_tensorflow_infer(image_filenames, image_f_coords)

In [34]:
test_image.shape

TensorShape([128, 128, 3])

In [35]:
test_image_for_inf = np.expand_dims(test_image, axis=0)
test_image_for_inf.shape

(1, 128, 128, 3)

In [36]:
#tf.config.run_functions_eagerly(False)
prediction = model1.predict(test_image_for_inf)

2025-09-30 16:21:15.845065: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node functional_1/batch_normalization_1/Cast/ReadVariableOp.
I0000 00:00:1759249276.259974    1312 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(da967389ca15b335:0:0), session_name()
I0000 00:00:1759249280.174227    1312 tpu_compile_op_common.cc:245] Compilation of da967389ca15b335:0:0 with session name  took 3.914210416s and succeeded
I0000 00:00:1759249280.186000    1312 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(da967389ca15b335:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_distributed_18313288938154636223", property.function_library_fingerprint = 3813490064238927170, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bou

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step


In [37]:
prediction.shape

(1, 14)

In [39]:
prediction

array([[3.9306428e-06, 2.1326291e-06, 5.8440691e-07, 7.5900585e-05,
        9.8279997e-06, 4.8590624e-05, 2.4825869e-05, 9.9963033e-01,
        4.6638044e-05, 5.7812360e-07, 8.5384585e-05, 3.9461293e-06,
        3.1830350e-05, 3.5500958e-05]], dtype=float32)

In [40]:
image_labels

7